# Explore hologram data Repeatability on PWV 

- author Sylvie Dagoret-Campagne
- creation date 2025-09-16 : version v1
- last update : 2025-09-24 : run version v5 : also compute statistical error on dPWV and ratio  dPWV/sigma and add  more plots 
- last update 
- affiliation : IJCLab
- Kernel @usdf **w_2025_36*
- Home emac : base (conda)
- laptop : conda_py313

**Goal** : Show Night variations of PWV wrt date and Time. Fit a straight line.

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figs_PWV02_PWVRepeatability"
prefix = "pwv02"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')


plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from astropy.coordinates.earth import EarthLocation
from datetime import datetime
from pytz import timezone

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit,least_squares


# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
from PWV00_parameters import *

In [ ]:
DumpConfig()

In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

### quartiles

In [ ]:
def q50(x):
    return x.quantile(0.5)

# 90th Percentile
def q90(x):
    return x.quantile(0.9)

def q25(x):
    return x.quantile(0.25)

def q75(x):
    return x.quantile(0.75)

def iqr(x):
    irq =  q75(x)-q25(x)
    return irq

#https://en.wikipedia.org/wiki/Interquartile_range    
#SD = IQR/1.35. for a normal distribution
def std_iqr(x):
    return iqr(x)/1.349
    
#df.groupby("AGGREGATE").quantile([0, 0.25, 0.5, 0.75, 0.95, 1])
#df.groupby("AGGREGATE").agg(("YOUR_COL_NAME", lambda x: x.quantile(0.5))

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

### Configuration

In [ ]:
observing_location = EarthLocation.of_site('Rubin Observatory')
tz = timezone('America/Santiago')

In [ ]:
FLAG_WITHCOLLIMATOR = True
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

In [ ]:
#version_results = "v2"
#legendtag = {"v1" : "v3.1.0","v2":"v3.1.0"}

In [ ]:
#atmfilenamesdict = {"v1":"../2025-06-26-SpectractorExtraction-FromButler/data/spectro/auxtel_atmosphere_20250912a_repomain_v1.npy",
#                    "v2":"../2025-06-26-SpectractorExtraction-FromButler/data/spectro/auxtel_run_20250917_w_2025_25_spectractorv31_holoallfilt_a_repomain_v1.npy",}

In [ ]:
#atmfilename = atmfilenamesdict[version_results]
#tag = legendtag[version_results] 

## Initialisation

### Read the file

- `atmfilename` is defined in `PW00_parameters.py` 

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

In [ ]:
#list(df_spec.columns)

### Compute NightObs and Select only after collimator

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
if FLAG_WITHCOLLIMATOR:
    df_spec = df_spec[df_spec["nightObs"]> DATE_WITHCOLLIMATOR]    

## Suppress Blue filters

In [ ]:
PWV_FILTER_LIST

In [ ]:
if FLAG_PWVFILTERS: 
    df_spec = df_spec[df_spec["FILTER"].isin(PWV_FILTER_LIST) ]
    # can't cut on empty filter because the time distance between pair would have to increase
    #df_spec = df_spec[df_spec["FILTER"].isin(['OG550_65mm_1']) ]

In [ ]:
# Compter le nombre d’entrées par nightObs et FILTER
counts = df_spec.groupby(["nightObs", "FILTER"]).size().unstack(fill_value=0)

# Plot en barres empilées
counts.plot(kind="bar", stacked=False, figsize=(18,6))

plt.ylabel("Number of Observations")
plt.xlabel("nightObs")
plt.title(f"Number of Observations per FILTER et per nightObs, {tag}")
plt.legend(title="FILTER")
plt.tight_layout()
plt.show()

#### Series on spec

In [ ]:
ser_spec_size = df_spec.groupby(["nightObs"]).size()

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(ser_spec_size.values,bins=50,facecolor="b")
ax.set_title("nb obs per night")
ax.set_xlabel("Nobs/night")

##### Make 3 series

In [ ]:
ser_CHI2_FIT = df_spec[["CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_PWV = df_spec[["PWV [mm]_x","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_PWV_CHI2_FIT = df_spec[["PWV [mm]_x","CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
#ser_PWV

In [ ]:
#ser_CHI2_FIT

In [ ]:
#ser_PWV_CHI2_FIT 

### Plot PWV and Chi2 from sereis before any selection

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="b",grid=True,title="Number of measurements per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Mean PWV per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Median PWV per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["std"].plot(kind='bar', subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="STD variation for PWV per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="r",grid=True,title="Number of measurements per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Mean CHI2 per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Median CHI2 per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["std"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="STD variation CHI2 per night")
plt.tight_layout()

In [ ]:
#ser_PWV_CHI2_FIT 

In [ ]:
#df_unstack = ser_PWV_CHI2_FIT.loc[20231010,:].unstack()
#df_unstack

In [ ]:
#ser_PWV_CHI2_FIT.loc[20231010,"PWV [mm]_x"]["count"]

In [ ]:
#df_unstack.loc["PWV [mm]_x","count"]

#### Add aggregate data added to pandas dataframe

In [ ]:
def FillAgreggates(row):
    the_nightObs = row["nightObs"]
    df_night = ser_PWV_CHI2_FIT.loc[the_nightObs,:].unstack()
    count = df_night.loc["PWV [mm]_x","count"]
    pwvmin = df_night.loc["PWV [mm]_x","min"]
    pwvmax = df_night.loc["PWV [mm]_x","max"]
    pwvmean = df_night.loc["PWV [mm]_x","mean"]
    pwvmedian = df_night.loc["PWV [mm]_x","median"]
    pwvstd = df_night.loc["PWV [mm]_x","std"]
    chi2min = df_night.loc["CHI2_FIT","min"]
    chi2max = df_night.loc["CHI2_FIT","max"]
    chi2mean = df_night.loc["CHI2_FIT","mean"]
    chi2median = df_night.loc["CHI2_FIT","median"]
    chi2std = df_night.loc["CHI2_FIT","std"]
    
    d = {"_count":count,"_pwvmin":pwvmin,"_pwvmax":pwvmax,"_pwvmean":pwvmean,"_pwvmedian":pwvmedian,"_pwvstd":pwvstd,
        "_chi2min":chi2min,"_chi2max":chi2max,"_chi2mean":chi2mean,"_chi2median":chi2median,"_chi2std":chi2std}
    ser = pd.Series(data=d)
    return ser

In [ ]:
row = df_spec.iloc[0]

In [ ]:
row

In [ ]:
FillAgreggates(row)

#### join dataframe + aggregates

In [ ]:
df_spec = df_spec.join(df_spec.apply(FillAgreggates,axis=1,result_type="expand"))

## Apply Quality selection

In [ ]:
fig,ax = plt.subplots(1,1)
df_spec["CHI2_FIT"].hist(bins=50,ax=ax,range=(0,200))
ax.set_yscale("log")

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(18,6))
ax1,ax2,ax3  = axs.flatten()
df_spec.hist("D_CCD [mm]_x",ax=ax1,bins=50,range=(DCCDMINFIG,DCCDMAXFIG),facecolor="b")
ax1.axvline(DCCDMINCUT,ls="-.",c="k")
ax1.axvline(DCCDMAXCUT,ls="-.",c="k")

df_spec.hist("CHI2_FIT",ax=ax2,bins=50,range=(0,500),facecolor="b")
#ax2.set_yscale("log")
ax2.axvline(CHI2CUT,ls="-.",c="k")


df_spec.hist("EXPTIME",ax=ax3,bins=20,facecolor="b")
ax3.axvline(EXPTIMECUT,ls="-.",c="k") 
plt.suptitle(tag)
plt.tight_layout()
plt.show()

### Add the Time in pd.datetime

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])

## Compute night boundaries

In [ ]:
def GetNightBoundariesDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of night boudaries
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        sel_flag = df_spec["nightObs"]== nightobs
        df_night = df_spec[sel_flag]
        tmin = df_night["Time"].min()-Dt
        tmax = df_night["Time"].max()+Dt
        d[nightobs] = (tmin,tmax)
    return d

In [ ]:
dn = GetNightBoundariesDict(df_spec)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("PWV [mm]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title(f"Precipitable water vapor measured by holo vs time before selection cuts , {tag}")

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

plt.tight_layout()

## Apply Quality selection cuts

- from parameter file

In [ ]:
cut = getSelectionCut(df_spec)

In [ ]:
df_spec_sel = df_spec[cut].drop(labels=['_count', '_pwvmin', '_pwvmax', '_pwvmean', '_pwvmedian', '_pwvstd','_chi2min', '_chi2max', '_chi2mean', '_chi2median', '_chi2std'],axis=1)

In [ ]:
df_spec_sel.reset_index(drop=True,inplace=True)

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

df_spec_sel.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("PWV [mm]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title(f"Precipitable water vapor measured by holo vs time after selection cuts , {tag}")

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

plt.tight_layout()

## Compute per-night aggregates

### Compute series per night on PWV and Chi2 fit quantities

In [ ]:
ser_PWV_CHI2_FIT_sel = df_spec_sel[["PWV [mm]_x","CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
ser_PWV_CHI2_FIT_sel.head()

In [ ]:
def FillAgreggatesSel(row):
    the_nightObs = row["nightObs"]

    # find the current nightObs and extract the aggregates 
    df_night = ser_PWV_CHI2_FIT_sel.loc[the_nightObs,:].unstack()
    count = df_night.loc["PWV [mm]_x","count"]
    pwvmin = df_night.loc["PWV [mm]_x","min"]
    pwvmax = df_night.loc["PWV [mm]_x","max"]
    pwvmean = df_night.loc["PWV [mm]_x","mean"]
    pwvmedian = df_night.loc["PWV [mm]_x","median"]
    pwvstd = df_night.loc["PWV [mm]_x","std"]
    chi2min = df_night.loc["CHI2_FIT","min"]
    chi2max = df_night.loc["CHI2_FIT","max"]
    chi2mean = df_night.loc["CHI2_FIT","mean"]
    chi2median = df_night.loc["CHI2_FIT","median"]
    chi2std = df_night.loc["CHI2_FIT","std"]
    
    d = {"_count":count,"_pwvmin":pwvmin,"_pwvmax":pwvmax,"_pwvmean":pwvmean,"_pwvmedian":pwvmedian,"_pwvstd":pwvstd,
        "_chi2min":chi2min,"_chi2max":chi2max,"_chi2mean":chi2mean,"_chi2median":chi2median,"_chi2std":chi2std}
    
    ser = pd.Series(data=d)

    return ser
    

In [ ]:
df_spec_sel = df_spec_sel.join(df_spec_sel.apply(FillAgreggatesSel,axis=1,result_type="expand"))

In [ ]:
TMIN_sel = df_spec_sel["nightObs"].min()
TMAX_sel = df_spec_sel["nightObs"].max()

## Recompute night boundaries

In [ ]:
dn = GetNightBoundariesDict(df_spec_sel)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

df_spec_sel.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg,ms=10)
ax.set_ylabel("PWV [mm]_x")
ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by holo selected vs time")

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

plt.tight_layout()
figname =f"{pathfigs}/pwv_allpoints_allnights"+figtype
plt.savefig(figname)
plt.show()

## Plot series on selected data

In [ ]:
ser_CHI2_FIT_sel = df_spec_sel[["CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_PWV_sel = df_spec_sel[["PWV [mm]_x","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="b",grid=True,title="Number of measurements per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Mean PWV per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Median PWV per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["std"].plot(kind='bar', subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="STD variation for PWV per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Mean CHI2 per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Median CHI2 per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["std"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="STD variation CHI2 per night after selection")
plt.tight_layout()

## Show time variation each night

In [ ]:
all_selected_nights = df_spec_sel["nightObs"].unique()

In [ ]:
def funclineres(params, x, y, yerr):
    # Return residual = fit-observed
    return (y-params[0] -params[1]*x)/yerr
def funcline(params,x):
    return params[0] + params[1]*x

In [ ]:
def MakeLineFit(df_night_pwv_curve):
    """
    """

    x = df_night_pwv_curve["dt"].values
    y = df_night_pwv_curve["PWV [mm]_x"].values
    yerr = df_night_pwv_curve["PWV [mm]_err_x"].values
    n = len(y)
        
    #popt, pcov = optimize.curve_fit(f, x, y, [1,-4])
    fit_res = least_squares(funclineres,[5.,0],args = (x,y,yerr))
    popt = fit_res.x 
    npar = len(popt)
    J = fit_res.jac
    cov = np.linalg.inv(J.T.dot(J))
    chi2dof = ((funclineres(popt,x,y,yerr))**2).sum()/(n-npar)
    cov *= chi2dof
    perr = np.sqrt(np.diagonal(cov)) 
    
    xfit = np.linspace(x.min()*0.99,x.max()*1.05)
    yfit = funcline(popt,xfit)
        
    slope = popt[1]
    slope_err = perr[1]

    return x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err

### Plot night by night

In [ ]:
df_spec_sel.head()

In [ ]:
List_Of_Nights = df_spec_sel['nightObs'].unique()
FirstNightNum = List_Of_Nights[0]

df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == FirstNightNum ]
#select the variables
df_night_pwv_curve = df_spec_night[["nightObs","Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","TARGET"]]
tmin = df_night_pwv_curve["Time"].min()
df_night_pwv_curve["dt"] = (df_night_pwv_curve["Time"] - tmin).dt.total_seconds()/3600.

In [ ]:
df_night_pwv_curve.iloc[0]

In [ ]:
def ComputeRepeatability(df):
    """
    Compute Repeatability of PWV for Spectrogram and and Spectrum night by night
    df is the dataframe containing the measurements of PWV of the whole night
    
    """
    # number of measurment for the night
    N = len(df)

    # return a dataframe  containing all pairs found
    # nightObs : night tag
    # dt : relative time if the first element of the pair 
    # dt_rep : time distance between the second element of the pair and the firt one
    # dPWVx_rep : difference in PWV Spectrogram
    # dPWVy_rep : difference in PWV Spectrum
    # targflag_rep : is the target the same ?
    
    dfout = pd.DataFrame(index=df.index,columns = ["nightObs","dt","dt_rep","dPWVx_rep","dPWVy_rep","targflag_rep","Npoints", "sig_dPWVx_rep", "sig_dPWVy_rep"])
    dfout["targflag_rep"].astype(bool)
    #dfout["Npoints"].astype(int)

    # initialise No previous target
    target_old = "No"
    time_old = 0.
    pwvx_old = 0.
    pwvy_old = 0.
    pwvx_err_old = 0.
    pwvy_err_old = 0.

    # loop on measurements of the night    
    for index in range(N):
        
        nightObs =  df.iloc[index]["nightObs"]

        # if first measurement
        if index == 0:
            # get the relative time of this first element of the pair
            dt0 = df.iloc[index]["dt"]
            dfout.iloc[index] = [ nightObs,dt0, 0., 0., 0., False,N, 0.,0.]
        else:
            # find the date and time of the second 
            target_new = df.iloc[index]["TARGET"]
            time_new = df.iloc[index]["dt"]
            pwvx_new = df.iloc[index]["PWV [mm]_x"]
            pwvy_new = df.iloc[index]["PWV [mm]_y"]
            pwvx_err_new = df.iloc[index]["PWV [mm]_err_x"]
            pwvy_err_new = df.iloc[index]["PWV [mm]_err_y"]
            
            flag_target = (target_new == target_old)
            dPWVx_rep = pwvx_new - pwvx_old
            dPWVy_rep = pwvy_new - pwvy_old
            dt_rep = (time_new-time_old)*3600. # in seconds
            sigma_dPWVx_rep = np.sqrt(pwvx_err_new**2 +  pwvx_err_old**2)
            sigma_dPWVy_rep = np.sqrt(pwvy_err_new**2 +  pwvy_err_old**2)
            
            dfout.iloc[index] = [ nightObs,time_new, dt_rep, dPWVx_rep, dPWVy_rep, flag_target,N, sigma_dPWVx_rep, sigma_dPWVy_rep]
        
        target_old = df.iloc[index]["TARGET"]
        time_old = df.iloc[index]["dt"]
        pwvx_old = df.iloc[index]["PWV [mm]_x"]
        pwvy_old = df.iloc[index]["PWV [mm]_y"]
        pwvx_err_old = df.iloc[index]["PWV [mm]_err_x"]
        pwvy_err_old = df.iloc[index]["PWV [mm]_err_y"]
        
    return dfout

In [ ]:
ComputeRepeatability(df_night_pwv_curve)

In [ ]:
all_df_repeatability = []

# loop on nights
for night in all_selected_nights:
    # Select the night
    df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == night]
    
    # Select the variables
    df_night_pwv_curve = df_spec_night[["nightObs","Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","TARGET"]]

    tmin = df_night_pwv_curve["Time"].min()

    # Convert in hours
    df_night_pwv_curve["dt"] = (df_night_pwv_curve["Time"] - tmin).dt.total_seconds()/3600.
    
    # Compute the repeatability on PWV x and y
    df_rep = ComputeRepeatability(df_night_pwv_curve)
    
    # Keep all repeatability
    all_df_repeatability.append(df_rep)
    
    cut_on_dt = (df_rep["dt_rep"]>0.) & (df_rep["dt_rep"]< 120.)   
    cut_on_target = df_rep["targflag_rep"]
    
    cut_loose = cut_on_dt
    cut_strong = cut_on_dt & cut_on_target 
        
    # plot  
    fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(12,4))
    
    leg1 = ax1.get_legend()
    leg2 = ax2.get_legend()
    
      
    df_rep[cut_strong]["dPWVx_rep"].hist(ax=ax1,bins=60,range=(-3,3),facecolor="b",legend=leg1,label="Same target")
    df_rep[cut_strong]["dPWVy_rep"].hist(ax=ax2,bins=60,range=(-3,3),facecolor="b")
    
    df_rep[cut_loose]["dPWVx_rep"].hist(ax=ax1,bins=60,range=(-3,3),color="r",lw=3,histtype="step")
    df_rep[cut_loose]["dPWVy_rep"].hist(ax=ax2,bins=60,range=(-3,3),color="r",lw=3,histtype="step")
    
    ax1.set_xlabel("dPWV (mm)")
    ax1.set_title("Spectrogram")
    ax2.set_xlabel("dPWV (mm)")
    ax2.set_title("1D-Spectrum")
    
    df_rep[cut_strong].plot.scatter(x="dPWVx_rep",y="dPWVy_rep",marker='.',c="b",ax=ax3)
    ax3.set_xlim(-3,3.)
    ax3.set_ylim(-3,3.)
    ax3.grid()
    ax3.set_xlabel("dPWV (2D) (mm)")
    ax3.set_ylabel("dPWV (1D) (mm)")
    
    
    title = f"PWV repeatability for Night {night}, {tag}"
    plt.suptitle(title) 
    plt.tight_layout()
    #figname =f"{pathfigs}/pwv_per_night_{night}"+figtype
    #    plt.savefig(figname)
    plt.show()

       

## Merge all repeatability dataFrame

In [ ]:
df_allrep = pd.concat(all_df_repeatability)

In [ ]:
df_allrep.head()

## Selection of acceptable pairs for repeatability

- pairs must be separated by less than 2 minutes
- enough pairs per night
- loose cut no restriction on same target
- strong cut : choose the same target

In [ ]:
cut_on_dt = (df_allrep["dt_rep"]> 0.) & (df_allrep["dt_rep"]< 120.) & (df_allrep["Npoints"] > 10.)     
cut_on_target = df_allrep["targflag_rep"]

In [ ]:
cut_loose = cut_on_dt
cut_strong = cut_on_dt & cut_on_target 

In [ ]:
dfl = df_allrep[cut_loose][["dt","dt_rep","dPWVx_rep","dPWVy_rep","sig_dPWVx_rep","sig_dPWVy_rep"]]
dfs = df_allrep[cut_strong][["dt","dt_rep","dPWVx_rep","dPWVy_rep","sig_dPWVx_rep","sig_dPWVy_rep"]]

In [ ]:
dfl["ratio_x"] = dfl["dPWVx_rep"]/dfl["sig_dPWVx_rep"]  
dfs["ratio_x"] = dfs["dPWVx_rep"]/dfs["sig_dPWVx_rep"]  

In [ ]:
dfl["ratio_x"] = pd.to_numeric(dfl["ratio_x"], errors="coerce")
dfl = dfl[np.isfinite(dfl["ratio_x"].values)]
dfs["ratio_x"] = pd.to_numeric(dfs["ratio_x"], errors="coerce")
dfs = dfs[np.isfinite(dfs["ratio_x"].values)]

### Compute Statistics

- apply statistics on strong criteria

In [ ]:
dfl_stat = dfl.aggregate(["count","mean","std",lambda x : std_iqr(x)])
dfl_stat = dfl_stat.rename(index={"<lambda>": "std_iqr"})

In [ ]:
dfs_stat = dfs.aggregate(["count","mean","std",lambda x : std_iqr(x)])
dfs_stat = dfs_stat.rename(index={"<lambda>": "std_iqr"})

In [ ]:
dfs_stat

#### First Step to compute ddPWVx_rep, ddPWVy_rep, in order to compute sigma_mad

In [ ]:
# strong cut
dfs_stat = dfl.aggregate(["count","mean","std",lambda x : std_iqr(x)])
dfs_stat = dfs_stat.rename(index={"<lambda>": "std_iqr"})

meanx = dfs_stat.loc["mean","dPWVx_rep"]
stdx = dfs_stat.loc["std","dPWVx_rep"]
#stdx_iqr = dfs_stat.loc["<lambda>","dPWVx_rep"]
stdx_iqr = dfs_stat.loc["std_iqr","dPWVx_rep"]

meany = dfs_stat.loc["mean","dPWVy_rep"]
stdy = dfs_stat.loc["std","dPWVy_rep"]
#stdy_iqr = dfs_stat.loc["<lambda>","dPWVy_rep"]
stdy_iqr = dfs_stat.loc["std_iqr","dPWVx_rep"]

#### add 2 columns centralize dPWVx_rep, dPWVy_rep --> ddPWVx_rep, ddPWVy_rep used to compute sigma_mad
dfs["ddPWVx_rep"] = np.abs(dfs["dPWVx_rep"] - meanx)
dfs["ddPWVy_rep"] = np.abs(dfs["dPWVy_rep"] - meany)

### recompute count, mean, std (not irq) to compute sigma_mad
dfs_stat = dfs.aggregate(["count","mean","std"])

meanx = dfs_stat.loc["mean","dPWVx_rep"]
stdx = dfs_stat.loc["std","dPWVx_rep"]
madx =  dfs_stat.loc["mean","ddPWVx_rep"]
sigma_madx = np.sqrt(np.pi/2.)*madx

#meany = dfs_stat.loc["mean","dPWVy_rep"]
#stdy = dfs_stat.loc["std","dPWVy_rep"]
mady =  dfs_stat.loc["mean","ddPWVy_rep"]
sigma_mady = np.sqrt(np.pi/2.)*mady

textstr_pwvx = "$\delta_{pwv}$ = "+ f"{meanx:.2f}" + "$\pm + $" + f"{stdx:.2f} mm" + "\n" + "$\\sigma_{MAD} = $" + f" {sigma_madx:.2f} mm"
textstr_pwvy = "$\delta_{pwv}$ = "+ f"{meany:.2f}" + "$\pm + $" + f"{stdy:.2f} mm" + "\n" + "$\\sigma_{MAD} = $" + f" {sigma_mady:.2f} mm"

In [ ]:
print(textstr_pwvx)
print(textstr_pwvy)

#### second Step : Compute ratio

In [ ]:
# strong cut
dfs_stat = dfs.aggregate(["count","mean","std",lambda x : std_iqr(x)])
dfs_stat = dfs_stat.rename(index={"<lambda>": "std_iqr"})

meanx = dfs_stat.loc["mean","dPWVx_rep"]
stdx = dfs_stat.loc["std","dPWVx_rep"]
stdx_iqr = dfs_stat.loc["std_iqr","dPWVx_rep"]

## Compute sigma and sigma_irq on ratio_x
stdx_ratio = dfs_stat.loc["std","ratio_x"]
stdx_iqr_ratio = dfs_stat.loc["std_iqr","ratio_x"]

meany = dfs_stat.loc["mean","dPWVy_rep"]
stdy = dfs_stat.loc["std","dPWVy_rep"]
stdy_iqr = dfs_stat.loc["std_iqr","dPWVy_rep"]

#### add 2 columns centralize dPWVx_rep, dPWVy_rep --> ddPWVx_rep, ddPWVy_rep used to compute sigma_mad
dfs["ddPWVx_rep"] = np.abs(dfs["dPWVx_rep"] - meanx)
dfs["ddPWVy_rep"] = np.abs(dfs["dPWVy_rep"] - meany)

### recompute count, mean, std (not irq) to compute sigma_mad
dfs_stat = dfs.aggregate(["count","mean","std", lambda x: std_iqr(x)])
dfs_stat = dfs_stat.rename(index={"<lambda>": "std_iqr"})

meanx = dfs_stat.loc["mean","dPWVx_rep"]
stdx = dfs_stat.loc["std","dPWVx_rep"]
stdx_iqr = dfs_stat.loc["std_iqr","dPWVx_rep"]
madx =  dfs_stat.loc["mean","ddPWVx_rep"]
sigma_madx = np.sqrt(np.pi/2.)*madx

meany = dfs_stat.loc["mean","dPWVy_rep"]
stdy = dfs_stat.loc["std","dPWVy_rep"]
stdy_iqr = dfs_stat.loc["std_iqr","dPWVy_rep"]
mady =  dfs_stat.loc["mean","ddPWVy_rep"]
sigma_mady = np.sqrt(np.pi/2.)*mady


In [ ]:
textstr_pwvx = "$\delta_{pwv}$ = "+ f"{meanx:.2f}" + "$\pm + $" + f"{stdx:.2f} mm" + "\n" + "$\\sigma_{MAD} = $" + f" {sigma_madx:.2f} mm" + "\n" + "$RMS = $" + f" {stdx:.2f} mm" + "\n" + "$\sigma_{IQR} = $" + f" {stdx_iqr:.2f} mm"
textstr_pwvy = "$\delta_{pwv}$ = "+ f"{meany:.2f}" + "$\pm + $" + f"{stdy:.2f} mm" + "\n" + "$\\sigma_{MAD} = $" + f" {sigma_mady:.2f} mm" + "\n" + "$RMS = $" + f" {stdy:.2f} mm" + "\n" + "$\sigma_{IQR} = $" + f" {stdy_iqr:.2f} mm"

In [ ]:
print(textstr_pwvx )

In [ ]:
print(textstr_pwvy)

In [ ]:
dfs_stat

In [ ]:
mean_r =  dfs_stat.loc["mean","ratio_x"]
std_r =  dfs_stat.loc["std","ratio_x"]
iqr_r =  dfs_stat.loc["std_iqr","ratio_x"]

In [ ]:
textstr_ratio_x = "$\delta_{pwv}/\sigma_{pwv}$ = "+ f"{mean_r:.1f}" + "$\pm + $" + f"{std_r:.1f}" + "\n" +  "$RMS = $" + f" {std_r:.1f}"  + "\n" +  "$\sigma_{irq} = $" + f" {iqr_r:.1f} " 

In [ ]:
print(textstr_ratio_x)

## Plot repeatability

In [ ]:
# plot  
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(12,5),layout='constrained')
    
leg1 = ax1.get_legend()
leg2 = ax2.get_legend()
     
histdata1 = df_allrep[cut_strong]["dPWVx_rep"].hist(ax=ax1,bins=60,range=(-3,3),facecolor="b",legend=leg1,label="Same target") 
df_allrep[cut_loose]["dPWVx_rep"].hist(ax=ax1,bins=60,range=(-3,3),color="r",lw=3,histtype="step")
ax1.set_xlabel("$\Delta$ PWV (mm)")

ax1.text(0.05, 0.95, textstr_pwvx, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax1.set_title("pwv pair difference")


ratio_s = df_allrep[cut_strong]["dPWVx_rep"].values/df_allrep[cut_strong]["sig_dPWVx_rep"].values
ratio_l = df_allrep[cut_loose]["dPWVx_rep"].values/df_allrep[cut_loose]["sig_dPWVx_rep"].values

ratio_s =ratio_s.astype(float)
ratio_l =ratio_l.astype(float)

ratio_s = ratio_s[np.isfinite(ratio_s)]
ratio_l = ratio_l[np.isfinite(ratio_l)]

ratio_s = ratio_s[~np.isnan(ratio_s)]
ratio_l = ratio_l[~np.isnan(ratio_l)]

histdata2 = ax2.hist(ratio_s,bins=60,facecolor="b",label="Same target",range=(-100,100))
ax2.hist(ratio_l,bins=60,color="r",lw=3,histtype="step",range=(-100,100))
ax2.set_xlabel("$\Delta PWV /\sigma_{PWV}$")
ax2.set_title("ratio of pair pwv difference to stat error")
ax2.grid()
ax2.text(0.05, 0.95, textstr_ratio_x, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)


title = f"PWV repetability ({TMIN_sel} - {TMAX_sel}), {tag}"
plt.suptitle(title,fontsize=12)
#ax2.legend()

figname =f"{pathfigs}/{prefix}_repeatability_allnights_single"+figtype
plt.tight_layout()
plt.savefig(figname)
plt.show()

In [ ]:
def std_iqr(x):
    q75, q25 = np.percentile(x.dropna(), [75, 25])
    iqr = q75 - q25
    return iqr / 1.349  # approx de l'écart-type


### Fitting

In [ ]:
counts2 = histdata2[0]
values2 = histdata2[1]
bin_centers2 = 0.5 * (values2[:-1] + values2[1:])
x2 = np.linspace(values2.min(),values2.max(),100)

In [ ]:
data1 = df_allrep[cut_strong]["dPWVx_rep"].values
histdata1 = np.histogram(data1,bins=60,range=(-3,3))

In [ ]:
counts1, values1 = histdata1
bin_centers1 = 0.5 * (values1[:-1] + values1[1:])
x1 = np.linspace(values1.min(),values1.max(),100)

In [ ]:
from astropy.modeling import models, fitting

In [ ]:
lorentz_model1 = models.Lorentz1D(amplitude=100.0, x_0=0., fwhm=0.5)
lorentz_model2 = models.Lorentz1D(amplitude=100.0, x_0=0., fwhm= 5)

In [ ]:
fitter = fitting.LevMarLSQFitter()

In [ ]:
lorentz_fit1 = fitter(lorentz_model1,bin_centers1,counts1)
lorentz_fit2 = fitter(lorentz_model2,bin_centers2,counts2)

In [ ]:
sig1 = lorentz_fit1.fwhm.value/2.36
sig2 = lorentz_fit2.fwhm.value/2.36
param_text1 = ("Lorentz fit parameters : \n"
              f"Ampl = {lorentz_fit1.amplitude.value:.2f} mm\n"
              f"Center = {lorentz_fit1.x_0.value:.2f} mm \n"
              f"FWHM = {lorentz_fit1.fwhm.value:.2f} mm \n"
              f"sigma = {sig1:.2f} mm"
              )

param_text2 = ("Lorentz fit parameters : \n"
              f"Ampl = {lorentz_fit2.amplitude.value:.2f}\n"
              f"Center = {lorentz_fit2.x_0.value:.2f}\n"
              f"FWHM = {lorentz_fit2.fwhm.value:.2f} \n"
              f"sigma = {sig2:.2f}"
              )


In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,5),layout='constrained')

ax1.scatter(bin_centers1,counts1,color="r")
ax1.plot(x1, lorentz_fit1(x1), 'r-', label='Fit Lorentzienne')
ax1.text(0.05, 0.95, param_text1, transform=ax1.transAxes,
         fontsize=12, verticalalignment='top',
         bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.7))
ax1.set_xlabel("$\Delta$ PWV (mm)")

ax2.scatter(bin_centers2,counts2, color="b")
ax2.plot(x2, lorentz_fit2(x2), 'b-', label='Fit Lorentzienne')
ax2.text(0.05, 0.95, param_text2, transform=ax2.transAxes,
         fontsize=12, verticalalignment='top',
         bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.7))

title = f"Fit of PWV repetability  with Lorentz ({TMIN_sel} - {TMAX_sel}), {tag}"
plt.suptitle(title,fontsize=12)

In [ ]:
# plot  
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,5),layout='constrained')
    
leg1 = ax1.get_legend()
leg2 = ax2.get_legend()
     
histdata1 = df_allrep[cut_strong]["dPWVx_rep"].hist(ax=ax1,bins=60,range=(-3,3),facecolor="b",legend=leg1,label="Same target") 
df_allrep[cut_loose]["dPWVx_rep"].hist(ax=ax1,bins=60,range=(-3,3),color="r",lw=3,histtype="step")
ax1.set_xlabel("$\Delta$ PWV (mm)")

ax1.text(0.05, 0.95, textstr_pwvx, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax1.set_title("pwv pair difference")

ax1.plot(x1, lorentz_fit1(x1), 'k-', lw=2,label='Fit Lorentzienne')
ax1.text(0.65, 0.95, param_text1, transform=ax1.transAxes,
         fontsize=12, verticalalignment='top',
         bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.7))


ratio_s = df_allrep[cut_strong]["dPWVx_rep"].values/df_allrep[cut_strong]["sig_dPWVx_rep"].values
ratio_l = df_allrep[cut_loose]["dPWVx_rep"].values/df_allrep[cut_loose]["sig_dPWVx_rep"].values

ratio_s =ratio_s.astype(float)
ratio_l =ratio_l.astype(float)

ratio_s = ratio_s[np.isfinite(ratio_s)]
ratio_l = ratio_l[np.isfinite(ratio_l)]

ratio_s = ratio_s[~np.isnan(ratio_s)]
ratio_l = ratio_l[~np.isnan(ratio_l)]

histdata2 = ax2.hist(ratio_s,bins=60,facecolor="b",label="Same target",range=(-100,100))
ax2.hist(ratio_l,bins=60,color="r",lw=3,histtype="step",range=(-100,100))
ax2.set_xlabel("$\Delta PWV /\sigma_{PWV}$")
ax2.set_title("ratio of pair pwv difference to stat error")
ax2.grid()
ax2.text(0.05, 0.95, textstr_ratio_x, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)

ax2.plot(x2, lorentz_fit2(x2), 'k-', lw=2 ,label='Fit Lorentzienne')
ax2.text(0.65, 0.95, param_text2, transform=ax2.transAxes,
         fontsize=12, verticalalignment='top',
         bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.7))


title = f"PWV repetability ({TMIN_sel} - {TMAX_sel}), {tag}"
plt.suptitle(title,fontsize=12)
#ax2.legend()

figname =f"{pathfigs}/{prefix}_repeatability_andfit_allnights_single"+figtype
plt.tight_layout()
plt.savefig(figname)
plt.show()